#### Idei:
* x Overfitting: schimbat dropout
* x Mai multe batch-uri pentru training online
* x Verificat ca ia corect screenshots pentru state si next state
* x Scoate actiunea "0"
* Verificat daca e un offset la stari atunci cand faci mutari conform modelului
* Verificat daca fullTarget updateaza cum trebuie doar actiunea potrivita
* Schimbare model (verificare cu MSE?)

Cea mai mare problema nu e ca tot incearca aceeasi actiune, ci ca nu invata sa evite peretele la ultima actiune

* ! Daca am probleme cu sarpele sarind pozitii, poate e de la functia makeMove si pot face board variabila globala, sa nu o mai caute de fiecare data

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
import io
from PIL import Image, ImageOps
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob

set_image_data_format('channels_first')

#### Parameters

In [15]:
STILL_ALIVE_REWARD = 1
DEAD_REWARD = -1000

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 180
RESIZE_HEIGHT = 137
READ_BATCH = 2

MOVES = 4
RIGHT = 0
DOWN = 1
LEFT = 2
UP = 3

SHOTS_FOLDER = 'data/shots/'

SAMPLE_ALIVE = 1
SAMPLE_DEAD = 5
SAMPLE_REWARD = 20

MAX_MEMORY = 500

GAMMA = 0.75
EPS = 1.0
MIN_EPS = 0.05

#### Browser functions

In [16]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard():        
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    try:
        board.click()
    except WebDriverException:
        print('Exception')
        return
    
    
def saveScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getScreen():
    ss = browser.get_screenshot_as_png()
    im = Image.open(io.BytesIO(ss))
    
    im = preprocImg(im)
    return np.asarray(im.convert("L"))
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def makeMove(m):
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)

    if m == RIGHT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == DOWN:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == LEFT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == UP:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

def makeRandomMove():
    r = int(MOVES * random.random())
    makeMove(r)
    
    return r

#### Preprocess functions

In [17]:
def getLastGameIndex(folder):
    '''Get the index of the last current game.'''
    files = os.listdir(folder)
    if len(files) == 0:
        return 0
    
    last = files[-1]
    
    return parseName(last)[0]


def getName(g, i, m, r):
    '''Form the file name from the board data.'''
    return str(g).zfill(3) + '_' + str(i).zfill(3) + '_' + str(m) + '_' + str(r) + '.png'


def parseName(fileName):
    '''Parse the board data from the file name.'''
    return [int(d) for d in fileName[:-4].split('_')]


def saveImage(arr, file):
    '''Save an image array to a file'''
    img = Image.fromarray(arr.astype(np.uint8))
    img.save(file)


def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)
        

def preprocImg(im):
    '''Preprocess a screenshot.'''     
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = Image.open(fileName)   
            im = preprocImg(im)
            im.save(fileName)

### Q model

#### Input functions

In [26]:
def readData():
    files = os.listdir(SHOTS_FOLDER)
    
    labels = np.zeros(len(files))    
    data = []
    
    for i, f in enumerate(files):
        im = Image.open(SHOTS_FOLDER + f)
        
        if f in history:
            labels[i] = history[f][2]
        else:
            labels[i] = STILL_ALIVE_REWARD
            
        data.append([np.asarray(im.convert("L"))])
        
    data = np.asarray(data)
    return data, labels


def sampleData(nSamples, 
               propD=SAMPLE_DEAD, 
               propR=SAMPLE_REWARD,
               propA=SAMPLE_ALIVE):
    '''Sample the input data according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(SHOTS_FOLDER)
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseName(f)[3]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamplesBasic(samples):
    '''Read samples for basic model, which just classifies if a board is finished or not.'''
    data = []
    labels = []
    
    for file in samples:
        im = Image.open(SHOTS_FOLDER + file)
        g, i, m, r = parseName(file)
        
        data.append([np.asarray(im.convert("L"))])
        labels.append(1 if r > 0 else 0)
        
    data = np.asarray(data)
    return data, labels


def getPreviousSample(file):
    '''Get the name of the previous sample.'''
    
    g, i, m, r = parseName(file)
    i -= 1
    
    if i > 0:        
        prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile      
    else:
        return -1


def getNextSample(file):
    '''Get the name of the next sample.'''
    
    g, i, m, r = parseName(file)
    i += 1
    
    prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')
    if len(prevFile) == 1:
        prevFile = prevFile[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile
    else:
        return -1
    
    
def readSample(file):
    '''Read a single sample as an array.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    label = 1 if r > 0 else 0
    
    return data, label
    
    
def readSampleMove(file):
    '''Read a single sample as an array, also returns the move.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    if r == -1000:
        label = -1
    elif r > 1:
        label = r / 100
    else:
        label = 0
    
    return data, m, label


def readBatchSamples(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.'''
    data = []
    labels = []
    
    for file in samples:        
        dataBatch = []
        
        d, label = readSample(file)
        dataBatch.append(d)
        
        # Get previous files
        prevFile = file
        for p in range(batch - 1):
            prevFile = getPreviousSample(prevFile)
            
            if prevFile == -1:
                break
                
            d, _ = readSample(prevFile)
            dataBatch.append(d)
        
        if prevFile != -1:
            dataBatch.reverse()
            data.append(dataBatch)
            labels.append(label)
        
    data = np.asarray(data)
    return data, labels   


def getStateBatches(file, batch):
    '''Get the image batches for the current and next state.'''
          
    s, _, _ = readSampleMove(file)
    
    # Get the image batch for the current state
    curBatch = []
    curBatch.append(s)
    
    # Get previous files
    prevFile = file
    for p in range(batch - 1):
        prevFile = getPreviousSample(prevFile)
        
        if prevFile == -1:
            break
            
        s, _ = readSample(prevFile)
        curBatch.append(s)
        
    if prevFile == -1:
        return -1, -1
    else:
        #Get the image batch for the next state
        nextBatch = []
        
        nextFile = getNextSample(file)
        if nextFile == -1:
            curBatch.reverse()
            return curBatch, curBatch
        
        nextState, _ = readSample(nextFile)        
        
        nextBatch.append(nextState)        
        nextBatch.extend(curBatch[:-1])
        
        curBatch.reverse()
        nextBatch.reverse()
        return curBatch, nextBatch


def readBatchSamplesMove(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.
    Also reads the moves.'''
    states = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):          
        _, a, r = readSampleMove(file)
        
        curBatch, nextBatch = getStateBatches(file, batch)
        if curBatch != -1:
            states.append(curBatch)
            actions.append(a)
            labels.append(r)
            nextStates.append(nextBatch)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
       
    # Scale labels
#     labels = np.asarray(labels).reshape(-1, 1)
#     scaler = StandardScaler()
#     scaler.fit(labels)
#     labels = scaler.transform(labels)
    
    return states, actions, labels, nextStates    
    
    
def getModelMoveInput(model, screens):
    '''Get the move from the model which takes moves as input.'''
    
    m = 0
    bestMove = 0
    bestScore = DEAD_REWARD - 1
    
    while m < MOVES:
        score = model.predict([np.asarray([screens]), np.asarray([m])])      
        
        print(m, score)
        if score > bestScore:
            bestScore = score
            bestMove = m
            
        m += 1        
    
    
def getModelMoveOutput(model, screens):
    '''Get the move from the model which produces move scores as outputs.'''
    res = model.predict(np.asarray([screens]))
    m = np.argmax(res)
    
    return m     
    
    
def moveString(m):
    if m == RIGHT:
        return 'right'
    elif m == DOWN:
        return 'down '
    elif m == LEFT:
        return 'left '
    elif m == UP:
        return 'up   '
    else:
        return 'none '
    
def getMoveOutputEps(model, screens, eps):
    '''Get the move from the model which produces move scores as outputs, with an epsilon probability for exploration.'''
    
    # If there is a model
    if model != False:        
        p = random.random()
        
        # If exploitation was chosen
        if p > eps:
            res = model.predict(np.asarray([screens]))
            m = np.argmax(res)
            print('Model:', moveString(m), end='\r')

            return m
   
    # If there is no model or exploration was chosen
    m = int(MOVES * random.random())
    print('Rando:', moveString(m), end='\r')
    
    return m 
    

#### DNN action output

Functional API, citeste batch-uri de imagini, considera actiunea ca output. 

In [19]:

def nn_model_actionoutput():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.5)(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.5)(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam')
    
    return model

#### Train model function

In [20]:

class Experience:
    '''Class for an experience.'''
    def __init__(self, state, action, label, nextState):
        self.state = state
        self.action = action
        self.label = label
        self.nextState = nextState
                        
            
def rememberExp(memory, exps):
    '''Add new experiences to the memory.'''
    for e in exps:
        if len(memory) == MAX_MEMORY:
            memory.pop(0)
        memory.append(e)


def sampleMemory(memory, nSamples, propD, propR, propA):
    '''Sample experiences from the memroy according to some proportions.
    
    Parameters
    ----------
    memory: the remembered experiences
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    nSamples = min(len(memory), nSamples)
        
    perc = []
    nd, na, nr = (0, 0, 0)
    for exp in memory:
        r = exp.label
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(memory, size=nSamples, replace=False, p=perc)
    return samples


def evaluateLabels(model, states, actions, labels, sample=0):
    
    if sample > 0:
        sample = min(sample, len(states))
        indexes = random.sample(range(len(states)), sample)
        states = states[indexes]
        actions = actions[indexes]
        labels = labels[indexes]
        
    res = model.predict(states)   
    
    sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
    meanSqErr = np.mean(sqErr)
    
    return meanSqErr


def trainModelOnline(memory, 
                     model,
                     batchSize=32, 
                     batches=1,
                     file='data/models/model.h5',
                     status=True
                     ):
    '''Train a model online, from the experience memory. 
    Can load or generate a new model. Saves the model at the end.
    
    Parameters
    ----------
    memory: the experience memory
    model: the model to be trained
    batchSize: the training batch size
    batches: number of batches to train on
    file: the model file, used for saving
    status: if true, print the status of the training
    '''
        
    # Sample experiences from the memory
    samples = sampleMemory(memory, batches * batchSize, 5, 10, 1)
    batchSize = min(batchSize, len(samples))

    
    # For each batch
    bStart = 0
    
    while bStart < len(samples):
        
        bStop = min(bStart + batchSize, len(samples))
    
        states = []
        targets = []
        
        # For each experience
        while bStart < bStop: 
            e = samples[bStart]
            target = e.label
            
            if e.label != DEAD_REWARD:
                target = e.label + GAMMA * np.amax(model.predict(np.asarray([e.nextState])))

            fullTarget = model.predict(np.asarray([e.state]))
            fullTarget[0][e.action] = target

            targets.append(fullTarget[0])
            states.append(e.state)
            
            bStart += 1

        # Fit the model with the batch
        targets = np.asarray(targets)
        model.fit(np.asarray(states), targets, epochs=1, verbose=0)

    #     print('MSE =', evaluateLabels(model, states, actions, labels, sample=50))
        model.save(file)
        
        bStart = bStop
                
    return model


def trainModelOffline(states, actions, labels, nextStates, 
                      epochs, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=False,
                      status=True
                     ):
    '''Train a model offline. 
    Can load or generate a new model. Saves the model after each epoch.
    
    Parameters
    ----------
    states: the list of batches of images representing the current states
    actions: the list of actions taken
    labels: the list of rewards obtained
    nextStates: the list of batches of images representing the next states
    epochs: the number of training epochs
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    '''
    
    if load:
        model = load_model(file)  
        print('Model loaded.')
    else:
        model = nn_model_actionoutput()

    dataSize = len(states)
    
    # For each epoch
    for i in range(epochs):
        if status:
            print('Epoch', i+1)
        
        time1 = time.time()
    
        batchStart = 0
        batchEnd = min(batchSize, dataSize)
    
        # For each batch
        while batchStart < batchEnd:

            targets = []
            
            b = batchStart
            # For each experience of the batch, recalculate the targets according to the Q algorithm
            while b < dataSize and b < batchEnd:
                # Get the features of the experience
                s = states[b : b+1]
                a = actions[b]
                l = labels[b]
                n = nextStates[b: b+1]
                
                target = l
                if l != DEAD_REWARD:
                    target = l + GAMMA * np.amax(model.predict(n))
                
                fullTarget = model.predict(s)
                fullTarget[0][a] = target
                
                targets.append(fullTarget[0])
                
                b += 1
            
            if status:
                print(batchStart, '/', dataSize, end='\r')
                
            # Fit the model with the batch
            targets = np.asarray(targets)
            model.fit(states[batchStart:batchEnd], targets, epochs=1, verbose=0)

            batchStart = batchEnd
            batchEnd = min(batchEnd + batchSize, dataSize)
          
        if status:  
            print(dataSize, '/', dataSize)
            print("  MSE = %.2f" % evaluateLabels(model, states, actions, labels, sample=50))
        model.save(file)
        
        time2 = time.time()
        
        if status:
            print("  Duration: %.2f s" % (time2- time1))
        
        
    return model

In [21]:
# states, actions, labels, nextStates = readBatchSamplesMove(sampleData(6000), READ_BATCH)
# s2, a2, l2, n2 = readBatchSamplesMove(sampleData(50), READ_BATCH)
# print(states.shape)

# model = trainModelOffline(states, actions, labels, nextStates, epochs=50, load=True, file='data/models/model_s6000_e211.h5')

### Play game

In [22]:
def play(games,
         train=False,
         useModel=False,
         saveImages=False,
         modelFile='data/models/model.h5',
         saveFolder=SHOTS_FOLDER):
    global browser
    
    e = EPS
    
    url = 'https://playsnake.org/'
    browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
#     browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
    browser.get(url) 

    START_GAME = getLastGameIndex(saveFolder) + 1
    gameIndex = START_GAME
    memory = []
    
    # Load model
    if useModel == True:
        curModel = load_model(modelFile)
    elif train == True:
        curModel = nn_model_actionoutput()
    else:
        curModel = False

    for g in range(games):

        # Select the level
        selectLevel(1)        
        while getState() == 'countdown':
            pass

        clickBoard()
        prevScreen = getScreen()

        score = 0
        screenIndex = 0
        time1 = time.time()
    
        state = []
        episode = []        
        finalScore = 0
        
        m = DOWN
        prevM = m
        
        while getState() == 'playing' or getState() == 'paused':            
            curScreen = getScreen()    
            screenIndex += 1    
            
            # Save the current screenshot
            if saveImages:
                saveImage(curScreen, saveFolder + 'prev.png')

            # Add the new experience to the episode
            nextState = [prevScreen, curScreen]
            if state and nextState:
                exp = Experience(state, prevM, r, nextState)
                episode.append(exp)
            state = nextState
        
            # Make the move     
            prevM = m     
            m = getMoveOutputEps(curModel, state, e)
        
            clickBoard()
            makeMove(m)
            clickBoard() 

            # Calculate the reward
            r = max(STILL_ALIVE_REWARD, getScore() - score) 
        
            # Update last board  
            if saveImages:
                fileName = getName(gameIndex, screenIndex, prevM, r)
                os.rename(saveFolder + 'prev.png', saveFolder + fileName)
            
            score = getScore()
            finalScore = max(finalScore, score)

            prevScreen = curScreen
        
        time2 = time.time()        
        print("%d: %.2f s, %d points" % (g, time2-time1, finalScore))
        
        # Change reward of last board
        if saveImages:
            g, i, m, r = parseName(fileName)
            os.rename(saveFolder + fileName, saveFolder + getName(g, i, m, DEAD_REWARD))
            
        # Add last experience
        nextState = [prevScreen, curScreen]
        if state and nextState:
            # prevM sau m ?!?!?!
            exp = Experience(state, prevM, DEAD_REWARD, nextState)
            episode.append(exp)
        state = nextState
        
    
        if train:     
            # Add the episode to the memory
            rememberExp(memory, episode)

            # Retrain the model
            curModel = trainModelOnline(memory, model=curModel, file=modelFile, batches=5)
        else:
            time.sleep(1)            
            
        gameIndex += 1
        if e > MIN_EPS:
            e = e * 0.995

    browser.quit()
    
    
    ### DEBUGGING
    return episode



In [ ]:
# Train online
# EPS = 0.4
# MIN_EPS = 0.05
# MAX_MEMORY = 1000
# episode = play(5, 
#      useModel=True, 
#      modelFile='data/models/model_o142.h5', 
#      train=True   
#     )

# Save images
play(1, 
     saveImages=True   
    )

# Play with model
# EPS = 0.0
# play(1, useModel=True, modelFile='data/models/model_o142.h5')

In [116]:
for i, e in enumerate(episode):
    for j, s in enumerate(e.state):
        saveImage(s, 'data/shots2/e' + str(i) + '_c' + str(j) + '_' + moveString(e.action) + '_' + str(e.label) + '.png')
    for j, s in enumerate(e.nextState):
        saveImage(s, 'data/shots2/e' + str(i) + '_n' + str(j) + '.png')

### Evaluation

In [ ]:
# Build pipeline

layers = []
layers.append(('nn', KerasClassifier(build_fn=nn_model_actionlast,
                                   epochs=5,
                                   batch_size=10,
                                   verbose=1)))
estimator = Pipeline(layers)


# Evaluate

data, actions, labels = readBatchSamplesMove(sampleData(200), 2)

labels = np.asarray(labels)
labels = labels.reshape((labels.size, 1))
actions = actions.reshape((actions.size, 1))
print(data.shape)
print(actions.shape)
print(labels.shape)
     
sKFold = StratifiedKFold(n_splits=2)
results = cross_val_score(estimator, [data, actions], labels, cv=sKFold, verbose=1, scoring='accuracy')

print("Results: %.5f (%.5f)" % (results.mean(), results.std()))